## Table of Contents
Edit Distance

[1 - Data Preprocessing]

process_data

get_count


     
get_probs

[2 - String Manipulations]

delete_letter

switch_letter
     
replace_letter
     
insert_letter

[3 - Combining the Edits]

Edit One Letter

Edit_two_letters

Suggest Spelling Suggestions-get_corrections

[4 - Minimum Edit Distance]
     
[Dynamic Programming]-min_edit_distance




<a name='0-1'></a>
Edit Distance

Models that correct words that are 1 and 2 edit distances away.
- We say two words are n edit distance away from each other when we need n edits to change one word into another.

An edit could consist of one of the following options:

- Delete (remove a letter): ‘hat’ => ‘at, ha, ht’
- Switch (swap 2 adjacent letters): ‘eta’ => ‘eat, tea,...’
- Replace (change 1 letter to another): ‘jat’ => ‘hat, rat, cat, mat, ...’
- Insert (add a letter): ‘te’ => ‘the, ten, ate, ...’

the four methods are used to implement an Auto-correct.
- To do so, you will need to compute probabilities that a certain word is correct given an input.



The goal of our spell check model is to compute the following probability:

$$P(c|w) = \frac{P(w|c)\times P(c)}{P(w)} \tag{Eqn-1}$$

The equation above is [Bayes Rule](https://en.wikipedia.org/wiki/Bayes%27_theorem).
- Equation 1 says that the probability of a word being correct $P(c|w) $is equal to the probability of having a certain word $w$, given that it is correct $P(w|c)$, multiplied by the probability of being correct in general $P(C)$ divided by the probability of that word $w$ appearing $P(w)$ in general.
- To compute equation 1, you will first import a data set and then create all the probabilities that you need using that data set.

<a name='1'></a>
## 1 - Data Preprocessing

In [ ]:
import re
from collections import Counter
import numpy as np
import pandas as pd


**process_data**

1) Reads in a corpus (text file)

2) Changes everything to lowercase

3) Returns a list of words.

In [ ]:
def process_data(file_name):
    words = []  # return this variable correctly

    # Open the file, read its contents into a string variable
    with open(file_name, 'r') as file:
        text = file.read()

    # Convert all letters to lower case
    text = text.lower()

    # Use regular expression to find all words
    words = re.findall(r'\b\w+\b', text)
    return words

In [ ]:
word_l = process_data('/content/shakespeare (1).txt')
vocab = set(word_l)  # this will be your new vocabulary
print(f"The first ten words in the text are: \n{word_l[0:10]}")
print(f"There are {len(vocab)} unique words in the vocabulary.")

The first ten words in the text are: 
['o', 'for', 'a', 'muse', 'of', 'fire', 'that', 'would', 'ascend', 'the']
There are 6116 unique words in the vocabulary.


** get_count**
- The dictionary's keys are words
- The value for each word is the number of times that word appears in the corpus.



In [ ]:
def get_count(word_l):
    word_count_dict = {}  # fill this with word counts

    for word in word_l:
        if word in word_count_dict:
            word_count_dict[word] += 1
        else:
            word_count_dict[word] = 1

    return word_count_dict

In [ ]:
word_count_dict = get_count(word_l)
print(f"There are {len(word_count_dict)} key values pairs")
print(f"The count for the word 'thee' is {word_count_dict.get('thee',0)}")

There are 6116 key values pairs
The count for the word 'thee' is 240


**get_probs**

Given the dictionary of word counts, compute the probability that each word will appear if randomly selected from the corpus of words.

$$P(w_i) = \frac{C(w_i)}{M} \tag{Eqn-2}$$
where

$C(w_i)$ is the total number of times $w_i$ appears in the corpus.

$M$ is the total number of words in the corpus.



In [ ]:
def get_probs(word_count_dict):

    probs = {}  # return this variable correctly

    # get the total count of words for all words in the dictionary
    total_count = sum(word_count_dict.values())

    # calculate the probability of each word
    for word, count in word_count_dict.items():
        probs[word] = count / total_count

    return probs

In [ ]:
probs = get_probs(word_count_dict)
print(f"Length of probs is {len(probs)}")
print(f"P('thee') is {probs['thee']:.4f}")

Length of probs is 6116
P('thee') is 0.0045


<a name='2'></a>
## 2 - String Manipulations

Now the computed $P(w_i)$ for all the words in the corpus,write functions to manipulate strings so that we can edit the erroneous strings and return the right spellings of the words.

* `delete_letter`: given a word, it returns all the possible strings that have **one character removed**.
* `switch_letter`: given a word, it returns all the possible strings that have **two adjacent letters switched**.
* `replace_letter`: given a word, it returns all the possible strings that have **one character replaced by another different letter**.
* `insert_letter`: given a word, it returns all the possible strings that have an **additional character inserted**.


Delete letter

In [ ]:
def delete_letter(word, verbose=False):

    delete_l = []
    split_l = []

    for i in range(len(word)):
        split_l.append((word[:i], word[i:]))
        delete_l.append(word[:i] + word[i+1:])

    if verbose: print(f"input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")

    return  delete_l

In [ ]:
delete_word_l = delete_letter(word="cans",
                        verbose=True)

input word cans, 
split_l = [('', 'cans'), ('c', 'ans'), ('ca', 'ns'), ('can', 's')], 
delete_l = ['ans', 'cns', 'cas', 'can']


Switch letter

In [ ]:
def switch_letter(word, verbose=False):

    switch_l = []
    split_l = []
    switch_l = [word[:i] + word[i+1] + word[i] + word[i+2:] for i in range(len(word) - 1)]
    split_l = [(word[:i], word[i:]) for i in range(len(word) - 1)]

    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}")
    return switch_l

In [ ]:
switch_word_l = switch_letter(word="eta",
                         verbose=True)

Input word = eta 
split_l = [('', 'eta'), ('e', 'ta')] 
switch_l = ['tea', 'eat']


Replace letter

In [ ]:
def replace_letter(word, verbose=False):
    letters = 'abcdefghijklmnopqrstuvwxyz'

    replace_l = []
    split_l = []

    for i in range(len(word)):
        for letter in letters:
            if letter != word[i]:  # Ensure we are actually replacing the letter
                replaced_word = word[:i] + letter + word[i+1:]
                replace_l.append(replaced_word)
                split_l.append((word[:i], word[i+1:]))

    # Sort the list, for easier viewing
    replace_l = sorted(replace_l)

    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")

    return replace_l

In [ ]:
replace_l = replace_letter(word='can',
                              verbose=True)

Input word = can 
split_l = [('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('', 'an'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('c', 'n'), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', ''), ('ca', '')] 
replace_l ['aan', 'ban', 'caa', 'cab', 'cac', 'cad', 'cae', 'caf', 'ca

Insert letter

In [ ]:
def insert_letter(word, verbose=False):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_l = []
    split_l = []

    for i in range(len(word) + 1):
        for letter in letters:
            inserted_word = word[:i] + letter + word[i:]
            insert_l.append(inserted_word)
            split_l.append((word[:i], letter, word[i:]))

    if verbose: print(f"Input word {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")
    return insert_l

In [ ]:
insert_l = insert_letter('at', True)
print(f"Number of strings output by insert_letter('at') is {len(insert_l)}")

Input word at 
split_l = [('', 'a', 'at'), ('', 'b', 'at'), ('', 'c', 'at'), ('', 'd', 'at'), ('', 'e', 'at'), ('', 'f', 'at'), ('', 'g', 'at'), ('', 'h', 'at'), ('', 'i', 'at'), ('', 'j', 'at'), ('', 'k', 'at'), ('', 'l', 'at'), ('', 'm', 'at'), ('', 'n', 'at'), ('', 'o', 'at'), ('', 'p', 'at'), ('', 'q', 'at'), ('', 'r', 'at'), ('', 's', 'at'), ('', 't', 'at'), ('', 'u', 'at'), ('', 'v', 'at'), ('', 'w', 'at'), ('', 'x', 'at'), ('', 'y', 'at'), ('', 'z', 'at'), ('a', 'a', 't'), ('a', 'b', 't'), ('a', 'c', 't'), ('a', 'd', 't'), ('a', 'e', 't'), ('a', 'f', 't'), ('a', 'g', 't'), ('a', 'h', 't'), ('a', 'i', 't'), ('a', 'j', 't'), ('a', 'k', 't'), ('a', 'l', 't'), ('a', 'm', 't'), ('a', 'n', 't'), ('a', 'o', 't'), ('a', 'p', 't'), ('a', 'q', 't'), ('a', 'r', 't'), ('a', 's', 't'), ('a', 't', 't'), ('a', 'u', 't'), ('a', 'v', 't'), ('a', 'w', 't'), ('a', 'x', 't'), ('a', 'y', 't'), ('a', 'z', 't'), ('at', 'a', ''), ('at', 'b', ''), ('at', 'c', ''), ('at', 'd', ''), ('at', 'e', ''), ('at'

<a name='3'></a>
## 3 - Combining the Edits



Edit One Letter

 To get all the possible edits that are one edit away from a word. The edits  consist of the replace, insert, delete, and optionally the switch operation.

In [ ]:
def edit_one_letter(word, allow_switches = True):
    edit_one_set = set()

    letters = 'abcdefghijklmnopqrstuvwxyz'

    # Deletions
    delete_l = [word[:i] + word[i+1:] for i in range(len(word))]

    # Replacements
    replace_l = [word[:i] + letter + word[i+1:] for i in range(len(word)) for letter in letters]

    # Insertions
    insert_l = [word[:i] + letter + word[i:] for i in range(len(word) + 1) for letter in letters]

    # Switches (if allowed)
    if allow_switches:
        switch_l = [word[:i] + word[i+1] + word[i] + word[i+2:] for i in range(len(word) - 1)]
    else:
        switch_l = []

    # Update edit_one_set with all generated words
    edit_one_set.update(delete_l + replace_l + insert_l + switch_l)

    # Remove the original word itself from the set
    edit_one_set.discard(word)

    # return this as a set and not a list
    return set(edit_one_set)

In [ ]:
tmp_word = "at"
tmp_edit_one_set = edit_one_letter(tmp_word)
# turn this into a list to sort it, in order to view it
tmp_edit_one_l = sorted(list(tmp_edit_one_set))

print(f"input word {tmp_word} \nedit_one_l \n{tmp_edit_one_l}\n")
print(f"The type of the returned object should be a set {type(tmp_edit_one_set)}")
print(f"Number of outputs from edit_one_letter('at') is {len(edit_one_letter('at'))}")

input word at 
edit_one_l 
['a', 'aa', 'aat', 'ab', 'abt', 'ac', 'act', 'ad', 'adt', 'ae', 'aet', 'af', 'aft', 'ag', 'agt', 'ah', 'aht', 'ai', 'ait', 'aj', 'ajt', 'ak', 'akt', 'al', 'alt', 'am', 'amt', 'an', 'ant', 'ao', 'aot', 'ap', 'apt', 'aq', 'aqt', 'ar', 'art', 'as', 'ast', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz', 'au', 'aut', 'av', 'avt', 'aw', 'awt', 'ax', 'axt', 'ay', 'ayt', 'az', 'azt', 'bat', 'bt', 'cat', 'ct', 'dat', 'dt', 'eat', 'et', 'fat', 'ft', 'gat', 'gt', 'hat', 'ht', 'iat', 'it', 'jat', 'jt', 'kat', 'kt', 'lat', 'lt', 'mat', 'mt', 'nat', 'nt', 'oat', 'ot', 'pat', 'pt', 'qat', 'qt', 'rat', 'rt', 'sat', 'st', 't', 'ta', 'tat', 'tt', 'uat', 'ut', 'vat', 'vt', 'wat', 'wt', 'xat', 'xt', 'yat', 'yt', 'zat', 'zt']

The type of the returned object should be a set <class 'set'>
Number of outputs from edit_one_letter('at') is 129


 Edit Two Letters



To get two edits on a word. To do so, you would have to get all the possible edits on a single word and then for each modified word, you would have to modify it again.


In [22]:
def edit_two_letters(word, allow_switches = True):
    edit_two_set = set()

   # Generate all possible words that are one edit away
    edit_one_set = edit_one_letter(word, allow_switches=allow_switches)

    # Generate all possible words that are two edits away
    for w in edit_one_set:
        if w:
            edit_two_set.update(edit_one_letter(w, allow_switches=allow_switches))

    # return this as a set instead of a list
    return set(edit_two_set)

In [23]:
tmp_edit_two_set = edit_two_letters("a")
tmp_edit_two_l = sorted(list(tmp_edit_two_set))
print(f"Number of strings with edit distance of two: {len(tmp_edit_two_l)}")
print(f"First 10 strings {tmp_edit_two_l[:10]}")
print(f"Last 10 strings {tmp_edit_two_l[-10:]}")
print(f"The data type of the returned object should be a set {type(tmp_edit_two_set)}")
print(f"Number of strings that are 2 edit distances from 'at' is {len(edit_two_letters('at'))}")

Number of strings with edit distance of two: 2654
First 10 strings ['', 'a', 'aa', 'aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag']
Last 10 strings ['zv', 'zva', 'zw', 'zwa', 'zx', 'zxa', 'zy', 'zya', 'zz', 'zza']
The data type of the returned object should be a set <class 'set'>
Number of strings that are 2 edit distances from 'at' is 7154


spelling corrections

In [28]:
def get_corrections(word, probs, vocab, n=2, verbose = False):

    suggestions = []
    n_best = []

     # Generate possible corrections
    suggestions = list(edit_one_letter(word).union(edit_two_letters(word)))

    # Filter suggestions to include only those in the vocabulary
    valid_suggestions = [word for word in suggestions if word in vocab]

    # Sort the valid suggestions by their probabilities in descending order
    n_best = sorted(valid_suggestions, key=lambda x: probs.get(x, 0), reverse=True)[:n]

    # Create tuples with word and probability
    n_best = [(word, probs.get(word, 0)) for word in n_best]

    if verbose: print("entered word = ", word, "\nsuggestions = ", suggestions)

    return n_best

In [29]:
my_word = 'dys'
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=True) # keep verbose=True
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")

# CODE REVIEW COMMENT: using "tmp_corrections" insteads of "cors". "cors" is not defined
print(f"data type of corrections {type(tmp_corrections)}")

entered word =  dys 
suggestions =  ['vyzs', 'dysbr', 'dbyse', 'apdys', 'dysaf', 'dki', 'ldysm', 'iym', 'dyysv', 'ndyt', 'wdyqs', 'xyw', 'uns', 'zmdys', 'dsa', 'adyw', 'dxwys', 'dyksz', 'dbm', 'idyw', 'dxays', 'edysv', 'dylsv', 'dysek', 'tldys', 'dfyp', 'yysi', 'hyws', 'edyse', 'dyfsi', 'dxxs', 'dpt', 'mysb', 'sts', 'dlym', 'dylsd', 'dyfls', 'ndysz', 'dyxsy', 'kdeys', 'zdwys', 'dsyys', 'dwo', 'cdyx', 'uas', 'uyu', 'dfse', 'dbc', 'dyid', 'dysfj', 'dnps', 'deyj', 'yyfs', 'dqv', 'dmh', 'dzoys', 'odcys', 'dyfas', 'dyjzs', 'dqyts', 'dyjqs', 'gdyss', 'fyo', 'lyks', 'zas', 'ddd', 'tdsys', 'dyvfs', 'qdyse', 'doss', 'sdyrs', 'wdas', 'syse', 'dwsj', 'dyvus', 'gysn', 'dcysv', 'dyske', 'kyw', 'dydo', 'dbz', 'pysy', 'orys', 'dfxs', 'dfc', 'jdlys', 'dzw', 'sdsy', 'hyes', 'ydyos', 'dystw', 'fyc', 'idysa', 'oyso', 'wdmys', 'fyus', 'odfs', 'zdysb', 'dkbs', 'dyfvs', 'gays', 'odyi', 'oyh', 'kdyq', 'dlx', 'epys', 'pyfs', 'qdyp', 'dqzys', 'yyi', 'dqyst', 'hyxs', 'dssu', 'ddeys', 'cdays', 'pdyd', 'cdysz', '

<a name='4'></a>
## 4 - Minimum Edit Distance


<a name='4-1'></a>
### 4.1 - Dynamic Programming

Dynamic Programming breaks a problem down into subproblems which can be combined to form the final solution. Here, given a string source[0..i] and a string target[0..j], we will compute all the combinations of substrings[i, j] and calculate their edit distance.

min_edit_distance



In [30]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):


      # initialize cost matrix with zeros and dimensions (m+1,n+1)
    m = len(source)
    n = len(target)
    D = np.zeros((m+1, n+1), dtype=int)

    # Initialize cost matrix
    for i in range(m+1):
        D[i, 0] = i * del_cost
    for j in range(n+1):
        D[0, j] = j * ins_cost

    # Fill in the rest of the matrix
    for i in range(1, m+1):
        for j in range(1, n+1):
            if source[i-1] == target[j-1]:
                cost = 0
            else:
                cost = rep_cost
            D[i, j] = min(D[i-1, j] + del_cost,
                          D[i, j-1] + ins_cost,
                          D[i-1, j-1] + cost)

    # Minimum edit distance
    med = D[m, n]
    return D, med


In [31]:
source =  'play'
target = 'stay'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list('#' + source)
cols = list('#' + target)
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  4 

   #  s  t  a  y
#  0  1  2  3  4
p  1  2  3  4  5
l  2  3  4  5  6
a  3  4  5  4  5
y  4  5  6  5  4


In [32]:
source =  'eer'
target = 'near'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list(source)
idx.insert(0, '#')
cols = list(target)
cols.insert(0, '#')
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  3 

   #  n  e  a  r
#  0  1  2  3  4
e  1  2  1  2  3
e  2  3  2  3  4
r  3  4  3  4  3
